In [1]:
import numpy as np 
import pandas as pd 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import wandb

# TODO
- See if ticket and cabin have any importance
- Try data imputation instead of fillna

# Dataset

In [2]:
df = pd.read_csv('/home/arjun/Desktop/GitHub/Kaggle-competitions/_Datasets/Titanic/train.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
y = df['Survived']

In [5]:
df = df.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [6]:
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [7]:
df['Sex'] = df['Sex'].replace({'male': 0, 'female': 1})
df['Embarked'] = df['Embarked'].replace({'S':1, 'C':2, 'Q':3})

/tmp/ipykernel_1191142/909058363.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Sex'] = df['Sex'].replace({'male': 0, 'female': 1})
/tmp/ipykernel_1191142/909058363.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Embarked'] = df['Embarked'].replace({'S':1, 'C':2, 'Q':3})


In [8]:
df.fillna(df.mean(), inplace=True)
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,0,22.0,1,0,7.2500,1.0
1,2,1,1,1,38.0,1,0,71.2833,2.0
2,3,1,3,1,26.0,0,0,7.9250,1.0
3,4,1,1,1,35.0,1,0,53.1000,1.0
4,5,0,3,0,35.0,0,0,8.0500,1.0


In [9]:
df_randomized = df.sample(frac=1).reset_index(drop=True)
train_df = df_randomized[:800]
test_df = df_randomized[800:]

In [10]:
test_df

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
800,62,1,1,1,38.000000,0,0,80.0000,1.362205
801,512,0,3,0,29.699118,0,0,8.0500,1.000000
802,628,1,1,1,21.000000,0,0,77.9583,1.000000
803,720,0,3,0,33.000000,0,0,7.7750,1.000000
804,67,1,2,1,29.000000,0,0,10.5000,1.000000
...,...,...,...,...,...,...,...,...,...
886,196,1,1,1,58.000000,0,0,146.5208,2.000000
887,337,0,1,0,29.000000,1,0,66.6000,1.000000
888,551,1,1,0,17.000000,0,2,110.8833,2.000000
889,163,0,3,0,26.000000,0,0,7.7750,1.000000


# Model Creation

In [11]:
num_epochs = 10000
lr =.001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

/home/arjun/Desktop/AI_ENV/lib/python3.10/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


device(type='cpu')

In [12]:
wandb.init(
    # set the wandb project where this run will be logged
    project="Titanic_with_NNs",

    # track hyperparameters and run metadata
    config={
    "learning_rate":lr,
    "epochs": num_epochs,
    }
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: arjun_g_ravi. Use `wandb login --relogin` to force relogin


In [13]:
class NeuralNet(nn.Module):
    def __init__(self, inp_size, h1, h2, h3,  out_size):
        super(NeuralNet, self).__init__()
        self.inp_size = inp_size
        self.lay1 = nn.Linear(inp_size, h1)
        self.lay2 = nn.ReLU()
        self.lay3 = nn.Linear(h1, h2)
        self.lay4 = nn.ReLU()
        self.lay5 = nn.Linear(h2, h3)
        self.lay6 = nn.ReLU()
        self.lay7 = nn.Linear(h3, out_size)
        self.drop1 = nn.Dropout(0.5)
        self.drop2 = nn.Dropout(0.1)
        
    def forward(self,x):
        out = self.lay1(x)
        out = self.lay2(out)
        out = self.lay3(out)
        out = self.drop1(out)
        out = self.lay4(out)
        out = self.lay5(out)
        # out = self.drop2(out)
        out = self.lay6(out)
        out = self.lay7(out) # We don't apply sotmax the cross entropy loss will do that for us
        return out
    
model = NeuralNet(8,30,20,30,8).to(device)

In [14]:
# class NeuralNet(nn.Module):
#     def __init__(self, inp_size, h1, h2, out_size):
#         super(NeuralNet, self).__init__()
#         self.fc1 = nn.Linear(inp_size, h1)
#         self.bn1 = nn.BatchNorm1d(h1)
#         self.drop1 = nn.Dropout(0.5)
#         self.fc2 = nn.Linear(h1, h2)
#         self.bn2 = nn.BatchNorm1d(h2)
#         self.drop2 = nn.Dropout(0.5)
#         self.fc3 = nn.Linear(h2, out_size)
        
#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.bn1(x)
#         x = nn.ReLU()(x)
#         x = self.drop1(x)
#         x = self.fc2(x)
#         x = self.bn2(x)
#         x = nn.ReLU()(x)
#         x = self.drop2(x)
#         x = self.fc3(x)
#         return x

# model = NeuralNet(8, 30, 15, 2).to(device)
    

In [15]:
lossCategory = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters(), lr = lr)

# Training Model

In [16]:
train_loader = DataLoader(dataset=torch.tensor(np.array(train_df)), shuffle=True, batch_size=1000)
test_loader = DataLoader(dataset=torch.tensor(np.array(test_df)), shuffle=True, batch_size=100)

In [17]:
def calc_train_accuracy():
    correct = 0
    total = 0
    for i, batch in enumerate(train_loader):
        X = torch.cat((batch[:, :1], batch[:, 2:]), dim=1).to(device).to(torch.float32)
        y = batch[:, 1].to(device).long()
        output = model(X)
        softmax_output = F.softmax(output, dim=-1)
        _, y_pred = torch.max(softmax_output, dim=-1)
        correct += (y_pred == y).sum().item()
        total += y.size(0)
    return correct / total * 100 # accuracy


def calc_test_accuracy():
    correct = 0
    total = 0
    for i, batch in enumerate(test_loader):
        X = torch.cat((batch[:, :1], batch[:, 2:]), dim=1).to(device).to(torch.float32)
        y = batch[:, 1].to(device).long()
        output = model(X)
        softmax_output = F.softmax(output, dim=-1)
        _, y_pred = torch.max(softmax_output, dim=-1)
        correct += (y_pred == y).sum().item()
        total += y.size(0)
    return correct / total * 100 # accuracy

In [18]:
for epoch in range(num_epochs):
    avg_loss = 0
    for i,batch in enumerate(train_loader):

        y = batch[:,1].to(device).long()
        X =torch.cat((batch[:, :1], batch[:, 2:]), dim=1).to(device).to(torch.float32)

        # Forward pass
        output = model(X)
        
        # Backward pass
        loss = lossCategory(output, y)
        loss.backward()
        optimiser.step()
        optimiser.zero_grad()
        avg_loss += loss
        train_acc = calc_train_accuracy()
        test_acc = calc_train_accuracy()
        wandb.log({"loss": loss, 'train acc':train_acc, 'test acc':test_acc})

    if not ((epoch+1)%1000): print(f"Epoch:{epoch+1}/{num_epochs} Loss:{avg_loss/len(train_loader)}")

Epoch:1000/10000 Loss:0.49984073638916016
Epoch:2000/10000 Loss:0.4099218249320984
Epoch:3000/10000 Loss:0.3910123109817505
Epoch:4000/10000 Loss:0.3885905146598816
Epoch:5000/10000 Loss:0.37916111946105957
Epoch:6000/10000 Loss:0.38364383578300476
Epoch:7000/10000 Loss:0.3691403865814209
Epoch:8000/10000 Loss:0.3578985631465912
Epoch:9000/10000 Loss:0.36407768726348877
Epoch:10000/10000 Loss:0.3375577926635742


# Checking Accuracy

In [19]:
wandb.finish()
calc_train_accuracy(), calc_test_accuracy()

loss,█▇▆▆▄▃▃▃▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▁▁▂▁▁▁▂▂▂▁▁▁▁▁
test acc,▁▂▃▄▆▆▇▇▇▇▇▇▇▇█▇▇▇▇███▇█████████████████
train acc,▁▂▂▄▅▆▆▇▇▇▇▇▇▇▇▇███▇▇▇█▇██▇█▇█▇███▇████▇
loss,0.33756
test acc,86.625
train acc,85.375


(85.75, 79.12087912087912)

# Inference on Test Dataset

In [20]:
df = pd.read_csv('/home/arjun/Desktop/GitHub/Kaggle-competitions/_Datasets/Titanic/test.csv')
df = df.drop(['Name', 'Ticket', 'Cabin'], axis=1)
df['Sex'] = df['Sex'].replace({'male': 0, 'female': 1})
df['Embarked'] = df['Embarked'].replace({'S':1, 'C':2, 'Q':3})
df.fillna(df.mean(), inplace=True)
ds = torch.tensor(np.array(df))
test_loader = torch.utils.data.DataLoader(dataset=ds, shuffle=False, batch_size=1)
ds.shape,df.shape

/tmp/ipykernel_1191142/2239982465.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Sex'] = df['Sex'].replace({'male': 0, 'female': 1})
/tmp/ipykernel_1191142/2239982465.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Embarked'] = df['Embarked'].replace({'S':1, 'C':2, 'Q':3})


(torch.Size([418, 8]), (418, 8))

In [21]:
l = []
id_ = []
for i,batch in enumerate(test_loader):
        X =batch.to(device).to(torch.float32)
        output = model(X)
        softmax_output = F.softmax(output, dim=-1)

        l.append(torch.argmax(softmax_output).item())
        id_.append(int(X[:,0].item()))

ans = pd.DataFrame({'PassengerId':id_,'Survived':l})
ans.to_csv('/home/arjun/Desktop/GitHub/Kaggle-competitions/_Datasets/Titanic/submission.csv', index=False)
print("Saved new O/P")
ans

Saved new O/P


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


# Best Score: 0.79